## 贝叶斯定理

$ P(XY)=P(X|Y)∗P(Y)=P(Y|X)∗P(X) $

------

## 朴素贝叶斯法

1. 设$ x=\{a_1,a_2,...,a_m\} $为一个待分类项，而$a_m$为x的一个特征属性
2. 有类别集合$ C = \{y_1,y_2,...,y_n\}$
3. 计算$ \max\limits_{y_i\in{C}}P(y_i|x)$
4. 根据贝叶斯定理$ \max\limits_{y_i\in{C}}P(y_i|x) = \max\limits_{y_i\in{C}}\cfrac{P(x|y_i)P(y_i)}{P(x)}$

如果x的各个特征属性是条件独立的，则$P(x)= \prod_{j=1}^m P(m_{a_m})$ ($P(m_{a_m})$ 表示第m个特征取值为$a_m$的概率)
通过上式可以看到，对于指定的x，P(x)是一个固定值,所以

1. 求$ \max\limits_{y_i\in{C}}P(y_i|x)$ 等价于求 $\max\limits_{y_i\in{C}} P(x|y_i)P(y_i)$
2. x的各个特征属性是条件独立,所以$P(x|y_i)P(y_i) = P(y_i) \prod_{j=1}^m P(m_{a_m}|y_i)$
3. $\prod_{j=1}^m P(m_{a_m}|y_i)$ 是小数连乘，结果会无限小，所以加上对数运算，不会改变max的值。
4. 按照3的描述，可得等价式：$\prod_{j=1}^m P(m_{a_m}|y_i)\iff\sum\limits_{j=1}^mlog_2P(m_{a_m}|y_i)$
5. 对于x的某个特征属性，可能存在$P(m_{a_m}|y_i)=0$ ，所以需要做拉普拉斯校准，即每个特征属性值对应的条数加1，总条数加m，这些校准的概率估计与对应的未校准的估计很接近，但是避免了零概率值。

上述假定x的各个特征属性是条件独立的，所以是朴素贝叶斯法。

In [10]:
from numpy import *


In [11]:

def loadDataSet():
    '''
    mock数据
    :return:
    '''
    postingList = [[' my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', ' to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]  # 1 代表 侮辱性 文字， 0 代表 正常 言论 return postingList, classVec
    return postingList, classVec


def createVocabList(dataSet):
    '''
    收集文档关键词全集
    :param dataSet:
    :return:
    '''
    flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]
    vocabSet = set(flatten(dataSet))
    return list(vocabSet)


def setOfWords2Vec(vocabList, inputSet):
    '''
    构建词组向量
    :param vocabList:
    :param inputSet:
    :return:
    '''
    return [(1 if w in inputSet else 0) for w in vocabList]

In [12]:
def trainNB0(trainMatrix, trainCategory):
    '''
    
    :param trainMatrix: 每一行代表一篇文档出现的单词向量(该向量的长度是所有文档的关键字全集长度)
    每一行都是用0/1表示文档中是否有某个单词
    :param trainCategory:  trainMatrix的长度应该与trainCategory的长度一致
    :return: 
    '''
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    # 侮辱性文档的先验概率
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num, p1Num = ones(numWords), ones(numWords)  # 拉普拉斯校准

    # 属于类别0和1文档的总词数
    p0Denom, p1Denom = 1.0, 1.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            # 拉普拉斯校准，所谓拉普拉斯平滑就是在计算类先验概率和属性条件概率时，在分子上则添加这个修正量与分类数目的乘积
            p1Denom += sum(trainMatrix[i])

        else:  # P(X=x|Y=c)=
            p0Num += trainMatrix[i]
            # 拉普拉斯校准
            p0Denom += sum(trainMatrix[i])

    # 连乘无限小，python表示为0，所以使用对数，然后求和（单调性和极值相同）
    p1Vect = log(p1Num / p1Denom)
    p0Vect = log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive

In [13]:
def classifyNBWithLn(vec2Classify, p0Vec, p1Vec, pClass1):
    '''
    这是二分类
    :param vec2Classify:
    :param p0Vec:
    :param p1Vec:
    :param pClass1: 侮辱文档的先验概率
    :return:
    '''
    # 是侮辱文档的后验概率
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0


In [14]:
def createTwoDimVec():
    '''
    训练出每个分类的最大似然估计的先验分布
    :return:
    '''
    listOPosts, listClasses = loadDataSet()
    allVocabVec = createVocabList(listOPosts)

    trainMat = [setOfWords2Vec(allVocabVec, example) for example in listOPosts]
    return allVocabVec, trainMat, listClasses


def testBayes():
    myVocabList, trainMat, listClasses = createTwoDimVec()
    p0V, p1V, pAb = trainNB0(trainMat, listClasses)

    testEntry = ['dog', 'stop', 'is']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as:', classifyNBWithLn(thisDoc, p0V, p1V, pAb))


testBayes()

['dog', 'stop', 'is'] classified as: 1
